In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.express as px
import logging

from prophet import Prophet
from sklearn.metrics import mean_squared_error
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
logging.getLogger('prophet').setLevel(logging.WARNING)
logging.getLogger('cmdstanpy').setLevel(logging.DEBUG)
logging.getLogger('cmdstanpy').setLevel(logging.WARNING)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data/Preenchido/Dados_Preenchidos.csv')

In [ ]:
df.head()

,Instituição,Cod.IBGE,UF,População,Mês,Conta,Valor,Ano,Data
0,Armação dos Búzios,3300233.0,RJ,31067.0,1,IPTU,2658875.0,2016,2016-01-01
1,Armação dos Búzios,3300233.0,RJ,31067.0,2,IPTU,6301063.0,2016,2016-02-01
2,Armação dos Búzios,3300233.0,RJ,31067.0,3,IPTU,1357260.0,2016,2016-03-01
3,Armação dos Búzios,3300233.0,RJ,31067.0,4,IPTU,1171114.0,2016,2016-04-01
4,Armação dos Búzios,3300233.0,RJ,31067.0,5,IPTU,429707.0,2016,2016-05-01


In [ ]:
df['Data'] = pd.to_datetime(df['Data'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5760 entries, 0 to 5759
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Instituição  5760 non-null   object        
 1   Cod.IBGE     5760 non-null   float64       
 2   UF           5760 non-null   object        
 3   População    5760 non-null   float64       
 4   Mês          5760 non-null   int64         
 5   Conta        5760 non-null   object        
 6   Valor        5760 non-null   float64       
 7   Ano          5760 non-null   int64         
 8   Data         5760 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(2), object(3)
memory usage: 405.1+ KB


In [ ]:
df_final = pd.DataFrame(columns=['Data','Valor','Instituição', 'Conta', 'Ano','Mês'])

resultados = {'Instituição': [],
              'Conta': [],
              'K': [],
              'MSE': []}

for cidade in df['Instituição'].unique():
  for imposto in df['Conta'].unique():
    df_cidade_imposto = df[(df['Instituição'] == cidade) & (df['Conta'] == imposto)].copy()
    df_cidade_imposto = df_cidade_imposto[['Data', 'Valor']]
    df_cidade_imposto = df_cidade_imposto.rename(columns={"Data": "ds", "Valor": "y"})

    for k in range(2,7):
      df_train = df_cidade_imposto.iloc[:k*12]
      df_val = df_cidade_imposto.iloc[k*12:]

      model = Prophet()
      model.fit(df_train)

      future = model.make_future_dataframe(periods=len(df_cidade_imposto) - len(df_train), freq='MS')  # 2 anos no futuro (frequência mensal)
      forecast = model.predict(future)[['ds', 'yhat']].iloc[k*12:]

      resultados['Instituição'].append(cidade)
      resultados['Conta'].append(imposto)
      resultados['K'].append(k)
      resultados['MSE'].append(mean_squared_error(df_val['y'], forecast['yhat']))

    df_resultados = pd.DataFrame(resultados)

    K = df_resultados[(df_resultados['Instituição'] == cidade) & (df_resultados['Conta'] == imposto)].sort_values(by='MSE').iloc[0]['K']

    df_train = df_cidade_imposto.iloc[:K*12]

    model = Prophet()
    model.fit(df_train)

    periods = (len(df_cidade_imposto) - len(df_train))+24
    future = model.make_future_dataframe(periods=periods, freq='MS')

    forecast = model.predict(future)[['ds', 'yhat']].iloc[-24:]
    forecast.rename(columns={'ds': 'Data', 'yhat': 'Valor'}, inplace=True)
    forecast['Instituição'] = cidade
    forecast['Conta'] = imposto
    forecast['Ano'] = forecast['Data'].dt.year
    forecast['Mês'] = forecast['Data'].dt.month

    df_cidade_imposto = df[(df['Instituição'] == cidade) & (df['Conta'] == imposto)][['Data','Valor','Instituição', 'Conta', 'Ano','Mês']]
    df_final = pd.concat([df_final, df_cidade_imposto, forecast], ignore_index=True)

DEBUG:cmdstanpy:input tempfile: /tmp/tmpndlz4o9t/5xj1obhm.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpndlz4o9t/0prxmdy2.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=81116', 'data', 'file=/tmp/tmpndlz4o9t/5xj1obhm.json', 'init=/tmp/tmpndlz4o9t/0prxmdy2.json', 'output', 'file=/tmp/tmpndlz4o9t/prophet_modelf5h9k6sg/prophet_model-20241022143635.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
14:36:35 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:36:35 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpndlz4o9t/u1xb_75q.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpndlz4o9t/_0rfzj0u.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local

In [ ]:
df_final['Valor'] = df_final['Valor'].round(2)
df_final['Ano'] = df_final['Ano'].astype(int)
df_final['Mês'] = df_final['Mês'].astype(int)

In [ ]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7200 entries, 0 to 7199
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Data         7200 non-null   datetime64[ns]
 1   Valor        7200 non-null   float64       
 2   Instituição  7200 non-null   object        
 3   Conta        7200 non-null   object        
 4   Ano          7200 non-null   int64         
 5   Mês          7200 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 337.6+ KB


In [ ]:
df_final.to_csv('/content/drive/MyDrive/data/Regressive/Dados_Regressao.csv', index=False)